In [2]:
# Фільтруємо дані за певними умовами
import urllib.request
import os
import time
from datetime import datetime
import pandas as pd
import re
import tabulate


In [3]:
def fetch_csv_data(country_code, start_year, end_year, data_type, save_path):
    provinces = range(1, 28)
    for province in provinces:
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country={country_code}&provinceID={province}&year1={start_year}&year2={end_year}&type={data_type}"
        
        attempt = 0
        success = False
        while attempt < 3 and not success:
            try:
                response = urllib.request.urlopen(url)
                content = response.read()
                success = True
            except urllib.error.URLError as err:
                attempt += 1
                print(f"[!] Error fetching province {province}: {err}")
                if attempt < 3:
                    print("Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"[!] Failed to download after 3 attempts.")
                    continue

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"NOAA_Province{province}_{timestamp}.csv"
        full_path = os.path.join(save_path, file_name)

        try:
            with open(full_path, 'wb') as csv_file:
                csv_file.write(content)
            print(f"[+] Saved: {file_name}")
        except IOError as io_err:
            print(f"[!] Write error: {io_err}")

def prepare_folder(folder_path):
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
        print("[+] Directory created.")

    user_input = input("Clear existing files in directory? (y/n): ").strip().lower()
    if user_input == 'y':
        for file in os.listdir(folder_path):
            try:
                os.remove(os.path.join(folder_path, file))
            except Exception as e:
                print(f"[!] Error deleting {file}: {e}")
        print("[+] Directory cleaned.")
    else:
        print("[+] Skipped cleaning.")

country = "UKR"
start_year = 1981
end_year = 2024
data_type = "Mean"
data_folder = "D:/AD/foranaconda/forlab2"

prepare_folder(data_folder)
fetch_csv_data(country, start_year, end_year, data_type, data_folder)

[+] Skipped cleaning.
[+] Saved: NOAA_Province1_20250506_194632.csv
[+] Saved: NOAA_Province2_20250506_194633.csv
[+] Saved: NOAA_Province3_20250506_194634.csv
[+] Saved: NOAA_Province4_20250506_194636.csv
[+] Saved: NOAA_Province5_20250506_194637.csv
[+] Saved: NOAA_Province6_20250506_194638.csv
[+] Saved: NOAA_Province7_20250506_194639.csv
[+] Saved: NOAA_Province8_20250506_194640.csv
[+] Saved: NOAA_Province9_20250506_194641.csv
[+] Saved: NOAA_Province10_20250506_194642.csv
[+] Saved: NOAA_Province11_20250506_194643.csv
[+] Saved: NOAA_Province12_20250506_194644.csv
[+] Saved: NOAA_Province13_20250506_194645.csv
[+] Saved: NOAA_Province14_20250506_194646.csv
[+] Saved: NOAA_Province15_20250506_194647.csv
[+] Saved: NOAA_Province16_20250506_194648.csv
[+] Saved: NOAA_Province17_20250506_194649.csv
[+] Saved: NOAA_Province18_20250506_194650.csv
[+] Saved: NOAA_Province19_20250506_194651.csv
[+] Saved: NOAA_Province20_20250506_194652.csv
[+] Saved: NOAA_Province21_20250506_194655.csv


In [4]:
def extract_text(html_string):
    tag_pattern = re.compile('<.*?>')
    return re.sub(tag_pattern, '', html_string)

def load_csv_file(file_path):
    column_names = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'extra']
    try:
        data = pd.read_csv(file_path, header=1, names=column_names, converters={'Year': extract_text})
        data.drop(data[data['VHI'] == -1].index, inplace=True)
        data.drop("extra", axis=1, inplace=True)
        data.drop(data.index[-1], inplace=True)
        return data
    except pd.errors.ParserError as error:
        print(f"[!] Problem reading file {file_path}: {error}")
        return None

def collect_data(folder_path):
    combined_data = []
    for csv_file in os.listdir(folder_path):
        full_path = os.path.join(folder_path, csv_file)
        if csv_file.endswith('.csv'):
            try:
                match = re.search(r'Province(\d+)_', csv_file)
                region_code = int(match.group(1)) if match else None

                content = load_csv_file(full_path)

                if content is not None and region_code not in [12, 20]:
                    content.insert(0, "PROVINCE_ID", region_code, True)
                    content = content.astype({"Year": int, "Week": int})
                    combined_data.append(content)

            except (ValueError, IndexError, AttributeError) as e:
                print(f"[!] Error processing {csv_file}: {e}")
    combined_data.sort(key=lambda x: int(x["PROVINCE_ID"].iloc[0]) if pd.notna(x["PROVINCE_ID"].iloc[0]) else 0)
    result = pd.concat(combined_data).drop_duplicates().reset_index(drop=True)
    return result


def display_table(data_frame, rows=10):
    print(tabulate.tabulate(data_frame.head(rows), headers='keys', tablefmt='heavy_grid', showindex=False))
    print(tabulate.tabulate(data_frame.tail(rows), headers='keys', tablefmt='heavy_grid', showindex=False))
save_location = "D:/AD/foranaconda/forlab2"
data_output = collect_data(save_location)
display_table(data_output)

┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃             1 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 ┃ 26.65 ┃ 34.06 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫

In [5]:
def extract_city_ids_from_csv(directory):
    all_city_data = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.endswith('.csv'):
            city_data = parse_city_ids(file_path)
            if city_data:
                all_city_data.extend(city_data)
    city_data = pd.DataFrame(all_city_data, columns=["City_ID", "City_Name"]).drop_duplicates(ignore_index=True)
    city_data.set_index('City_ID', inplace=True)

    return city_data

def parse_city_ids(file_path):
    city_info = []

    try:
        with open(file_path, 'r') as file:
            for line in file:
                matches = re.findall(r"=\s*(\d+):\s*([\w'\-\s]+),", line)
                city_info.extend(matches)
    except Exception as e:
        print(f"[!] Помилка зчитування {file_path}: {e}")

    return [(int(city_id), city_name.strip()) for city_id, city_name in city_info]
directory_path = "D:/AD/foranaconda/forlab2"
city_data = extract_city_ids_from_csv(directory_path)
print(city_data)


                City_Name
City_ID                  
2               Chernihiv
14               Luhans'k
27               Zhytomyr
15                  L'viv
26           Zaporizhzhya
4                  Crimea
5         Dnipropetrovs'k
1                Cherkasy
10         Khmel'nyts'kyy
11                   Kiev
8                 Kharkiv
13             Kirovohrad
3              Chernivtsi
12              Kiev City
24              Vinnytsya
17                 Odessa
25                  Volyn
16              Mykolayiv
9                 Kherson
18                Poltava
19                  Rivne
7        Ivano-Frankivs'k
6                Donets'k
21                   Sumy
20            Sevastopol'
23         Transcarpathia
22              Ternopil'


In [6]:
def remap_province_ids(data_frame):
    province_mapping = {
        16: 13, 27: 5, 10: 21, 3: 23, 11: 9, 25: 2, 17: 14, 21: 17, 1: 22,
        18: 15, 6: 4, 9: 20, 8: 19, 19: 16, 23: 6, 7: 8, 5: 3, 26: 7, 2: 24,
        4: 25, 15: 11, 22: 18, 13: 10, 24: 1, 14: 12
    }
    
    updated_data_frame = data_frame.copy()
    updated_data_frame.replace({"PROVINCE_ID": province_mapping}, inplace=True)
    
    return updated_data_frame

save_path = "D:/AD/foranaconda/forlab2"
data_output = collect_data(save_path)
data_mapped = remap_province_ids(data_output)

display_table(data_mapped)


┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   PROVINCE_ID ┃   Year ┃   Week ┃   SMN ┃    SMT ┃   VCI ┃   TCI ┃   VHI ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      1 ┃ 0.053 ┃ 260.31 ┃ 45.01 ┃ 39.46 ┃ 42.23 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      2 ┃ 0.054 ┃ 262.29 ┃ 46.83 ┃ 31.75 ┃ 39.29 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      3 ┃ 0.055 ┃ 263.82 ┃ 48.13 ┃ 27.24 ┃ 37.68 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      4 ┃ 0.053 ┃ 265.33 ┃ 46.09 ┃ 23.91 ┃ 35    ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫
┃            22 ┃   1982 ┃      5 ┃ 0.05  ┃ 265.66 ┃ 41.46 ┃ 26.65 ┃ 34.06 ┃
┣━━━━━━━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━╋━━━━━━━╋━━━━━━━┫

In [7]:
# Побудова графіку для аналізу
def get_vhi_data_extremes(region_code, start_year, end_year, data):
    region_names = {
        1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
        6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
        11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
        16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
        21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
    }
    region_name = region_names.get(region_code, "Невідомий регіон")
    filtered_data = data[(data["PROVINCE_ID"] == region_code) & (data["Year"] >= start_year) & (data["Year"] <= end_year)]
    vhi_data_values = filtered_data['VHI']
    valid_vhi_data_values = vhi_data_values[vhi_data_values != -1]
    
    if not valid_vhi_data_values.empty:
        min_vhi_data = valid_vhi_data_values.min()
        max_vhi_data = valid_vhi_data_values.max()
    else:
        min_vhi_data = None
        max_vhi_data = None
    return [region_code, region_name, min_vhi_data, max_vhi_data]

def process_vhi_data_data():
    folder_path = "D:/AD/foranaconda/forlab2"
    raw_data = collect_data(folder_path)          
    cleaned_data = remap_province_ids(raw_data) 
    
    start_year = 2024
    end_year = 2024
    
    result_data = []
    columns = ["Region Code", "Region Name", "Min VHI", "Max VHI"]

    for region_code in range(1, 26):  
        result_data.append(get_vhi_data_extremes(region_code, start_year, end_year, cleaned_data)) 
    print(tabulate.tabulate(result_data, headers=columns, tablefmt="simple"))
process_vhi_data_data()

  Region Code  Region Name          Min VHI    Max VHI
-------------  -----------------  ---------  ---------
            1  Вінницька              28.83      54.56
            2  Волинська              39.94      62.34
            3  Дніпропетровська       19         52.22
            4  Донецька               24.36      50.33
            5  Житомирська            37.66      56.83
            6  Закарпатська           37.49      61.48
            7  Запорізька             20.31      47.59
            8  Івано-Франківська      40.11      56.71
            9  Київська               33.41      55.4
           10  Кіровоградська         21.08      49.51
           11  Луганська              39.9       58.22
           12  Львівська              24.69      50.97
           13  Миколаївська           22.31      55.45
           14  Одеська                28.42      68.43
           15  Полтавська             23.9       57.31
           16  Рівенська              42.49      60.24
           

In [8]:
def get_vhi_data_extremes(provinces, year, data):
    region_names = {1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
        6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
        11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
        16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
        21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'}
    
    PROVINCE_NAME = region_names[provinces]
    
    filtered_data = data[(data["PROVINCE_ID"] == provinces) & (data["Year"] == year)]
    vhi_data_values = filtered_data['VHI']     
    vhi_data_values_filtered = vhi_data_values[vhi_data_values != -1]
    if not vhi_data_values_filtered.empty:
            min_value = vhi_data_values_filtered.min()
            max_value = vhi_data_values_filtered.max()    
    else:
        min_value = None
        max_value = None
    
    return [provinces, PROVINCE_NAME, year, min_value, max_value, vhi_data_values.tolist()[:10]]  

def main_analysis():
    save_path = "D:/AD/foranaconda/forlab2"
    data_output = collect_data(save_path)
    data_mapped = remap_province_ids(data_output)
    
    year = 2020
    provinces = 10

    vhi_data_analysis_results = []
    headers = ["Province ID", "PROVINCE_NAME", "Year", "Min VHI", "Max VHI", "VHI"]
    
    for provinces in range(1, 26):  
        vhi_data_analysis_results.append(get_vhi_data_extremes(provinces, year, data_mapped))
    
    print(tabulate.tabulate(vhi_data_analysis_results, headers=headers, tablefmt="simple"))
main_analysis()

  Province ID  PROVINCE_NAME        Year    Min VHI    Max VHI  VHI
-------------  -----------------  ------  ---------  ---------  ----------------------------------------------------------------------
            1  Вінницька            2020      34.48      64.12  [40.92, 43.19, 44.74, 45.29, 44.8, 43.92, 43.1, 42.88, 43.71, 45.61]
            2  Волинська            2020      39.48      61.93  [48.95, 49.23, 49.29, 49.4, 49.75, 50.26, 51.12, 51.59, 51.52, 51.72]
            3  Дніпропетровська     2020      19.4       68.14  [47.23, 46.99, 46.92, 46.55, 45.87, 45.14, 43.96, 43.39, 43.41, 43.13]
            4  Донецька             2020      17.88      59.14  [44.72, 46.28, 47.98, 48.67, 48.03, 46.47, 43.85, 42.14, 40.76, 39.92]
            5  Житомирська          2020      35.05      58.33  [42.82, 43.69, 44.52, 45.71, 45.62, 45.13, 45.76, 46.88, 47.05, 46.88]
            6  Закарпатська         2020      38.94      68.67  [39.29, 39.08, 38.94, 39.47, 40.11, 41.64, 44.34, 47.42, 50.0

In [9]:
from tabulate import tabulate

PROVINCE_NAME_dict = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
    6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
    11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
    16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'
}

def fetch_filtered_data(provinces, start_year, end_year, data):
    return data[(data["PROVINCE_ID"] == provinces) & (data["Year"] >= start_year) & (data["Year"] <= end_year)]

def extract_vhi_data_values(filtered_data):
    return filtered_data['VHI'].tolist()[:10]

def get_province_name(provinces):
    return PROVINCE_NAME_dict.get(provinces, "Unknown")

def gather_vhi_data_extremes(provinces, start_year, end_year, data):
    filtered_data = fetch_filtered_data(provinces, start_year, end_year, data)
    vhi_data_values = extract_vhi_data_values(filtered_data)
    return [provinces, get_province_name(provinces), start_year, end_year, vhi_data_values]

def calculate_extreme_years(data, threshold_percentage, VHI):
    years = sorted(data['Year'].unique())
    total_provinces = len(data['PROVINCE_ID'].unique())
    
    extreme_years = [
        year for year in years if check_drought_condition(year, data, VHI, threshold_percentage, total_provinces)
    ]
    return extreme_years

def check_drought_condition(year, data, VHI, threshold_percentage, total_provinces):
    condition = (data['Year'] == year) & (data['VHI'] <= VHI[1]) & (data['VHI'] >= VHI[0]) & (data['VHI'] != -1)
    drought_count = data[condition].groupby('PROVINCE_ID').size().count()
    return (drought_count / total_provinces) * 100 > threshold_percentage

def print_vhi_data_extremes(data):
    start_year, end_year = 1981, 2024
    result_data = [
        gather_vhi_data_extremes(provinces, start_year, end_year, data) for provinces in range(1, 11)
    ]
    
    headers = ["Province ID", "PROVINCE_NAME", "Year_Start", "Year_End", "VHI"]
    print("\n\t\t--- VHI Extremes per Province ---\n")
    print(tabulate(result_data, headers=headers, tablefmt="simple"))

def analyze_drought(data):
    VHI_extreme = [0, 15]
    extreme_years = calculate_extreme_years(data, 19, VHI_extreme)
    
    print("\n\n\t\t--- Extreme Drought Years ---")
    print(f"Years with extreme drought affecting more than 20% of provinces: {', '.join(map(str, extreme_years))}")
    
    VHI_moderate = [15, 35]
    moderate_years = calculate_extreme_years(data, 20, VHI_moderate)
    
    print("\n\n\t\t--- Moderate Drought Years ---")
    print(f"Years with moderate drought affecting more than 20% of provinces: {', '.join(map(str, moderate_years))}")

def last_f():    
    directory = "D:/AD/foranaconda/forlab2"
    data_frames_1 = collect_data(directory)
    data_frames_replace = remap_province_ids(data_frames_1)

    print_vhi_data_extremes(data_frames_replace)
    analyze_drought(data_frames_replace)

last_f()


		--- VHI Extremes per Province ---

  Province ID  PROVINCE_NAME        Year_Start    Year_End  VHI
-------------  -----------------  ------------  ----------  ----------------------------------------------------------------------
            1  Вінницька                  1981        2024  [45.9, 45.34, 44.88, 41.6, 39.29, 37.65, 35.03, 34.46, 35.04, 34.33]
            2  Волинська                  1981        2024  [53.88, 50.97, 45.65, 40.04, 36.44, 35.14, 33.57, 33.23, 33.53, 35.07]
            3  Дніпропетровська           1981        2024  [51.36, 47.14, 42.35, 38.53, 36.19, 34.71, 33.35, 33.65, 34.17, 34.27]
            4  Донецька                   1981        2024  [55.79, 51.93, 47.95, 43.2, 40.98, 40.76, 39.84, 37.98, 36.65, 35.87]
            5  Житомирська                1981        2024  [49.5, 48.62, 48.86, 46.28, 43.61, 41.71, 38.74, 37.61, 36.98, 36.44]
            6  Закарпатська               1981        2024  [54.44, 51.13, 45.73, 39.98, 34.6, 31.15, 30.7, 30.09, 2